## Issues

### Issues with Timestamp

- 11:01:00 - 11:01:59: only the minute was recorded --> done
- 14:20:01 - 14:40:30: the cell format is incorrect, only numerical values of a certain format are stored 
- 16:00:00 - 17:00:00: recorded in 12-hour format (e.g. 4:00:00 PM) --> DONE
- 13:59:46: recorded twice --> done
- 12:08:00 - 12:08:59: every other time stamp is missing --> 

### Issues with PVs

- 12:11:01 - 12:16:06: the unit of PT102/OUT.CV changed from barg to mPa (1 barg ~~ 0.1 mPa)
- 15:14:00 - 15:16:00: measurement error in FIC103/PID1/PV; FIC103/PID1/PV is different from FIC103/PID1/SP and FT103/OUT.CV.
- Missing values scattered in the data: connectivity error, nan, empty cells; some values (e.g. missing value in FT104/OUT.CV can be infered from FIC104/PID1/PV)
- Duplicated entries: "PT101/OUT.CV" at column E and "Pressure measure" at column Q

In [1]:
from pipeline.dataset import Dataset
df_dirty = Dataset('data/real/00-complete/dirty_data.csv')

In [2]:
df_dirty.infer_format()

'%H:%M:%S'

In [3]:
print(df_dirty.time_format)

%H:%M:%S


In [4]:
df_dirty.apply_format()

In [5]:
df_dirty.clean_stamps()

['11:01:00' '0x0']
Changing duplicate timestamp 11:01:00
11:00:59
Changing duplicate timestamp 0x0
14:20:00


In [62]:
df_dirty.df = df_dirty.df.drop_duplicates()
df_dirty.df.reset_index(drop=True, inplace=True)

In [69]:
 #Get List of all duplicated indices
df_i_dup = df_dirty.df[df_dirty.df.duplicated(subset=['TIMESTAMP'],keep=False)]
unique_dup = df_i_dup['TIMESTAMP'].unique() #[11:01:00, 0x0, 0x1]
print(unique_dup)
i=1

['11:01:00' '0x0']


In [70]:
tmp_df_i_dup = df_i_dup.loc[df_i_dup['TIMESTAMP'] == unique_dup[i]] #Get list of specific duplicated indices

In [71]:
n = len(tmp_df_i_dup)
n

1230

In [72]:
dup_index = tmp_df_i_dup[~tmp_df_i_dup.duplicated(subset=['TIMESTAMP'],keep='first')].index[0]
dup_index

15572

In [73]:
tmp_df_i_dup[~tmp_df_i_dup.duplicated(subset=['TIMESTAMP'],keep='first')]

,TIMESTAMP,Sm3/h,Sm3/h.1,C,barg,kg/s,kg/s.1,C.1,kg/m3,barg.1,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,barg.2
15572,0x0,120.419403,1.709,17.77886,1.251551,-0.001601,0.095191,23.469509,996.7639771,1.287201,0.0,0.000289,-0.001601,21.421841,0.1,0.095191,1.251551


In [75]:
df_dirty.df.iloc[dup_index]

TIMESTAMP              0x0
Sm3/h           120.419403
Sm3/h.1              1.709
C                 17.77886
barg              1.251551
kg/s             -0.001601
kg/s.1            0.095191
C.1              23.469509
kg/m3          996.7639771
barg.1            1.287201
Unnamed: 10            0.0
Unnamed: 11       0.000289
Unnamed: 12      -0.001601
Unnamed: 13      21.421841
Unnamed: 14            0.1
Unnamed: 15       0.095191
barg.2            1.251551
Name: 15572, dtype: object

In [12]:
from dateutil import parser
after = df_dirty.df.iloc[dup_index+n]['TIMESTAMP']
after = parser.parse(after)

In [14]:
before =df_dirty.df.iloc[dup_index-2]['TIMESTAMP']
before

'14:20:00'

In [25]:
before = parser.parse(before)

In [26]:
diff = after - before

In [27]:
diff /= (n+1)
diff

datetime.timedelta(seconds=1)

In [161]:

before_stamp

11:00:59


datetime.datetime(2021, 5, 8, 11, 0, 59)

In [173]:
y=4

In [174]:
curr_stamp = before_stamp + (y+1)*diff
#curr_stamp += diff
curr_stamp = curr_stamp.strftime(df_dirty.time_format)
curr_stamp

'11:01:04'

In [166]:
print(y*diff)
print(before_stamp)

0:00:00
2021-05-08 11:00:59


In [16]:
import os
cwd = os.getcwd()
cwd

'F:\\time_series_pipeline'

In [6]:
df_dirty.df.to_csv(cwd+'/output.csv', index=False)
#df_dirty.to_csv(cwd+'/output.csv')

NameError: name 'cwd' is not defined